# Personal Productivity Agent — Kaggle-style notebook UI
This notebook demonstrates running your agent pipeline interactively (like a Kaggle notebook).  
It loads tasks, runs the orchestrator, and prints a human-readable daily schedule.


In [7]:
import sys, os
ROOT = os.path.abspath("../")  # go 1 level up to project root
sys.path.append(ROOT)

print("PYTHONPATH updated:", ROOT)


PYTHONPATH updated: /home/bisagn/Documents/DhruviKava/PythonPractice/MyProjects/personal-productivity-agent-updated


In [8]:
# Cell 2 - bootstrap
import asyncio, json, os, time
from datetime import datetime
from IPython.display import Markdown, display, HTML, JSON

# Import orchestrator (run notebook from repo root)
from src.agents.orchestrator import create_orchestrator


ORCH = create_orchestrator()


{"timeout": 3600, "event": "session_service_initialized", "logger": "session_manager", "level": "info", "timestamp": "2025-11-18T11:06:48.892694Z"}
{"path": "./data/memory_bank.json", "event": "no_existing_memory", "logger": "memory_bank", "level": "info", "timestamp": "2025-11-18T11:06:48.895873Z"}


{"path": "./data/memory_bank.json", "event": "memory_bank_initialized", "logger": "memory_bank", "level": "info", "timestamp": "2025-11-18T11:06:48.897039Z"}
{"event": "plan_evaluator_initialized", "logger": "plan_evaluator", "level": "info", "timestamp": "2025-11-18T11:06:48.900795Z"}
{"event": "time_estimator_tool_initialized", "logger": "time_estimator", "level": "info", "timestamp": "2025-11-18T11:06:48.910482Z"}
{"event": "habit_analyzer_tool_initialized", "logger": "habit_analyzer", "level": "info", "timestamp": "2025-11-18T11:06:48.914471Z"}
{"max_tokens": 4000, "event": "context_engineer_initialized", "logger": "context_engineer", "level": "info", "timestamp": "2025-11-18T11:06:48.919852Z"}
{"base_dir": "/home/bisagn/Documents/DhruviKava/PythonPractice/MyProjects/personal-productivity-agent-updated/data/outputs", "event": "mcp_file_tool_initialized", "logger": "mcp_tools", "level": "info", "timestamp": "2025-11-18T11:06:48.924950Z"}
{"model": "gemini-2.0-flash-exp", "event": "c

In [9]:
def run_orchestrator(payload):
    session_id = f"nb_session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    return asyncio.run(ORCH.process_tasks(raw_tasks=payload, session_id=session_id))

def print_summary_from_result(result):
    # Try Reminder summary first
    try:
        summary = result["outputs"]["reminders"].get("summary")
        if summary:
            print(summary)
            return
    except Exception:
        pass
    # fallback: pretty print planned
    display(JSON(result["outputs"]["planned"]))


In [10]:
#Cell 4 — Load tasks & run (example)**

# Load existing tasks.json and run
tasks = load_tasks_from_file("data/tasks.json")
print("Loaded tasks:", len(tasks.get("tasks", [])))
res = run_orchestrator(tasks)
print("Workflow status:", res["workflow_status"])
print("Duration (ms):", int(res["total_duration_ms"]))


NameError: name 'load_tasks_from_file' is not defined

In [ ]:
print_summary_from_result(res)


In [ ]:
# Run a single quick task from a string
payload = {"tasks":[{"name":"Quick: Finish unit tests", "category":"coding","priority":"high","estimated_duration":45,"deadline":None}]}
res2 = run_orchestrator(payload)
print_summary_from_result(res2)


In [ ]:
# Save outputs to notebook outputs directory
outdir = "data/outputs/notebook_demo"
os.makedirs(outdir, exist_ok=True)
with open(os.path.join(outdir, f"result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"), "w") as f:
    json.dump(res2, f, indent=2)
print("Saved to:", outdir)
